# 0. what for
1.
# 1. preparations
## 1.1 全局设置

In [13]:
ARTIFICIAL = False

# Small dataset
settings_small_NN_path = r"../data/small_settings_NN.csv"
prod_embedding_small_path = "../data/small_prod_embedding_4.csv"

# Large data
settings_large_NN_path = r'E:\DATA\large_dta\large_settings_NN.csv'
prod_embedding_large_path = "E:\DATA\large_dta\large_prod_embedding_4.csv"

LEN = 8

# output path
train_root_path= "../data/train_8_all/"
train_file_head = "train_data_NP_"
train_file_tail= ".csv"

# for SA
# initial params
table_5_M = [0.025,3.72]
# lower/ upper bound
lb = [-0.3,0.01]
ub = [0.3,18]

import numpy as np
import csv
import pandas as pd
from visdom import Visdom
from tqdm import tqdm
from sko.tools import set_run_mode
from func_timeout import func_set_timeout
import func_timeout
import time
import os


## 1.2 read

In [14]:
data_key_small = pd.read_csv(settings_small_NN_path, encoding="utf-8")
data_key_large = pd.read_csv(settings_large_NN_path, encoding="utf-8")
data_key = pd.concat([data_key_small,data_key_large],axis=0,ignore_index=True)
len_small = data_key_small.shape[0]
len_large = data_key_large.shape[0]
# 有`N_uniq_auction`组setting
N_uniq_auction= data_key.shape[0]
print("For PT model, there are *{}* settings waiting to be inferred.".format(N_uniq_auction))
data_key.head()

For PT model, there are *1276* settings waiting to be inferred.


,desc,bidincrement,bidfee,retail,flg_endprice
0,Sony Ericsson S500i Unlocked Mysterious Green,0.15,0.75,499.99,0
1,PSP Slim & Lite Sony Piano Black,0.15,0.75,169.99,0
2,iPod Touch Apple 8GB with Software Upgrade,0.15,0.75,299.99,0
3,Logitech Cordless Wave Keyboard and Mouse,0.15,0.75,89.99,0
4,SanDisk Cruzer Contour 4GB,0.15,0.75,59.99,0


In [15]:
prod_embedding_small = pd.read_csv(prod_embedding_small_path, encoding="utf-8")
prod_embedding_large = pd.read_csv(prod_embedding_large_path, encoding="utf-8")
prod_embedding = pd.concat([prod_embedding_small,prod_embedding_large],axis=0,ignore_index=True)

- 合并data，去掉desc

In [16]:
prod_embedding.drop(columns=['desc'],inplace=True)
data_key.drop(columns=['desc'],inplace=True)

data = pd.concat([data_key,prod_embedding],axis=1)
data.head()

,bidincrement,bidfee,retail,flg_endprice,0,1,2,3
0,0.15,0.75,499.99,0,0.117575,0.014913,0.027262,-0.075999
1,0.15,0.75,169.99,0,0.060650,0.030211,-0.006635,0.039294
2,0.15,0.75,299.99,0,-0.043197,-0.076048,-0.119416,0.044113
3,0.15,0.75,89.99,0,0.001636,0.022876,-0.001473,0.074411
4,0.15,0.75,59.99,0,0.015891,-0.016824,-0.062114,-0.015273


In [17]:
data.shape

(1276, 8)

# 2. 拼接与输出: training data

In [18]:
LEN = 8
train_col = [str(i) for i in range(0,LEN)]

for i in tqdm(range(N_uniq_auction)):

    train_tmp = pd.DataFrame(data.iloc[i,:]).T
    # 重命名，避免合并出问题
    train_tmp.columns = train_col
    # Save
    output_path = train_root_path + train_file_head + str(i).zfill(4) + train_file_tail
    train_tmp.to_csv(output_path,header=True,index=False,encoding="utf-8")


100%|██████████| 1276/1276 [00:00<00:00, 1782.09it/s]


# 3. target data：只保留N<=T的target data

In [20]:
# input output path
if ARTIFICIAL:
    target_origin_root_path= r"../data/artificial_targets/"
    target_root_head= "../data/artificial_targets_ls_T/target_data_NP_"
else:
    target_origin_root_path= r"../data/targets_all/"
    target_root_head= "../data/targets_all_ls_T/target_data_NP_"

# output path
target_file_tail= ".csv"
all_target_file = os.listdir(target_origin_root_path)

In [21]:
for i in range(data_key.shape[0]):
# for i in range(1):
    input_target = os.path.join(target_origin_root_path,all_target_file[i])
    output_path = target_root_head + str(i).zfill(4)+target_file_tail

    settings = data_key.iloc[i,:]
    v = settings['retail']
    b = settings['bidfee']
    d = settings['bidincrement']
    if d==0:
        T = np.inf
    else:
        T = np.floor((v-b)/d)

    input = pd.read_csv(input_target,encoding="utf-8")
    output = input[input.N<=T]
    output.to_csv(output_path, encoding="utf-8", header=True, index=False)
print("Done")

Done
